In [32]:
#импорт необходимых библиотек и модулей
import os
import logging
import telegram
import re
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, CallbackQueryHandler
from google.cloud import dialogflow

#настройки dialogflow
updater = Updater("5748761513:AAHaCuFWeIXSvoDIF2QZovJecdSZCLsYBcg", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'aermolaevbot-fxea-e6f27deedf60.json'# скачaнный JSON


DIALOGFLOW_PROJECT_ID = 'aermolaevbot-fxea' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'AErmolaevbot'  # ID бота из телеграмма

In [33]:
#обработка команды старт (создаем Inline клавиатуру)
def startCommand(update: Update, context: CallbackContext):
    buttonA = telegram.InlineKeyboardButton('Получить тему курсовой работы', callback_data='buttomA')
    buttonB = telegram.InlineKeyboardButton('Поговорить', callback_data='buttomB')
    buttonE = telegram.InlineKeyboardButton('Прогноз погоды', callback_data='buttomE')
    markup = telegram.InlineKeyboardMarkup(inline_keyboard = [[buttonA], [buttonB], [buttonE]])
    
    update.message.reply_text('Добрый день! Чтобы начать работу, выберите одно из возможных действий', 
                              reply_markup = markup)
    return callback
    
#обработка нажатия клавиш клавиатуры
def callback(update: Update, context: CallbackContext):
    query = update.callback_query
    variant = query.data
    if variant == 'buttomA':
        query.answer()
        query.edit_message_text(text='О чем хотите писать курсовую? (форма ответа: "Курсовая: примерная тема")')
    
    if variant == 'buttomB':
        query.answer()
        query.edit_message_text('Давайте поговорим')
    
    if variant == 'buttomE':
        query.answer()
        query.edit_message_text(text='Напишите: "Город: свой город"')

## Прогноз погоды

In [34]:
#импорт необходимых библиотек и модулей
from geopy import geocoders
from yaweather import Russia, YaWeather

#функция получения погоды
def get_weather(city: str):
    geolocator = geocoders.Nominatim(user_agent="telebot")
    latitude = geolocator.geocode(city).latitude
    longitude = geolocator.geocode(city).longitude
    y = YaWeather(api_key='f5fc7043-3196-49ed-9848-02c19a3ba8e4')
    res = y.forecast((latitude, longitude))
    forecast = f'cейчас {res.fact.temp} °C, oщущается как {res.fact.feels_like} °C'
    return forecast

#печать прогноза погоды
def print_weather(update: Update, context: CallbackContext):
    city = update.message.text
    forecast = get_weather(re.sub(r'Город: ','',city))
    update.message.reply_text(f'Погода в городе {city[7:]}: {forecast}', parse_mode='Markdown')

## Болталка

In [35]:
#болталка
def detect_intent_texts(project_id, session_id, text, language_code):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(project_id, session_id)
    
    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)

    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
        
    return response.query_result.fulfillment_text

def textMessage(update: Update, context: CallbackContext):
  
    text_user = update.message.text
       
    text_bot = detect_intent_texts(DIALOGFLOW_PROJECT_ID, SESSION_ID, text_user, DIALOGFLOW_LANGUAGE_CODE)
    if text_bot:
        update.message.reply_text(text_bot)
    else:
        update.message.reply_text('Что?')

## Получение темы курсовой

In [36]:
#импорт необходимых библиотек и модулей
import pandas as pd
import string
import annoy
import pickle
import numpy as np
import tqdm 
from tqdm import tqdm_notebook
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

In [37]:
#препроцессинг текста
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

#0. датасет с темами
df = pd.read_excel('theme.xlsx')

#1. Функция преобразования темы (лемматизация, удаление стоп-слов и пунктуации)
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

#2. Процедура подготовки данных к обучению (создание списка токенов для каждой темы)
assert True
tweet_tokens = []

for text in df['Theme']:
    tweet_prep = preprocess_txt(text)
    tweet_tokens.append(tweet_prep)
    
#3. Функция поиска 5 ближайших тем
def get_response_theme(tweet, index, model, index_map):
    tweet_token = preprocess_txt(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet_token:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    similar_tweet = index.get_nns_by_vector(vector, 5)
    
    result = [index_map[i] for i in similar_tweet]
    return result

#4. Процедура поиска ближайшей темы
def get_model_index(model):
    model_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

    index_map = {}
    counter = 0

    for tweet in df['Theme']:
        n_model = 0
        index_map[counter] = tweet
        tweet_prep = preprocess_txt(tweet)
        
        vector_model = np.zeros(300) #вектор твита

        for word in tweet_prep:
            if word in model:
                vector_model += model[word]
                n_model += 1
        if n_model > 0:
            vector_model = vector_model / n_model #нормализуем вектор твита
       
        model_index.add_item(counter, vector_model) #добавляем элемент для индексации
     
        counter += 1

        if counter > 100000:
            break

    model_index.build(10) #из индексов создаем лес из 10 деревьев
    return model_index, index_map

modelW2V = Word2Vec(sentences=tweet_tokens, vector_size=300, window=5, min_count=1)

w2v_index, index_map = get_model_index(modelW2V.wv)

#вывод темы курсовой работы
def text_theme(update: Update, context: CallbackContext):
    txt_q = update.message.text # текст который пришёл
    answer = get_response_theme(re.sub(r'Курсовая: ','',txt_q), w2v_index, modelW2V.wv, index_map)
    update.message.reply_text(f'Ваша тема: {answer[0]}', parse_mode='Markdown')

## Хендлеры

In [ ]:
# Хендлеры
start_command_handler = CommandHandler('start', startCommand)
theme_handler = MessageHandler(Filters.regex('Курсовая: '), text_theme)
weather_handler = MessageHandler(Filters.regex('Город: '), print_weather)
callback_buttom_handler = CallbackQueryHandler(callback = callback, pattern=None, run_async=False)
                                               
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(theme_handler)
dispatcher.add_handler(weather_handler)
# Начинаем поиск обновлений
updater.start_polling()
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()

Exiting immediately!
